# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [56]:
# import libraries
import pandas as pd
import numpy as np
import sqlite3
import re

from sklearn.datasets import make_multilabel_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier

from sqlalchemy import create_engine

import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [57]:
def load_data():
    # load data from database
    engine = create_engine('sqlite:///DisasterDB_ETL.db')
    conn = engine.connect()
    df = pd.read_sql_table("DisasterETL_DB", conn)
    X = df['message'] #only the message text, and not others.
    Y= df.drop(['id','genre','message','original'],axis=1)
    category_names = list(y.columns) #Y should only contain category colunm
    X_head = X.head(5)
    Y_head = Y.head(5)
    
    return X, Y, category_names 


In [58]:
X, y, category_names = load_data()


### 2. Write a tokenization function to process your text data

In [59]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [60]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)



In [61]:
def build_model():
    #As an estimator, we'll implement Support Vector Classifier
    svc = SVC(gamma="scale")

    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',MultiOutputClassifier( RandomForestClassifier()))
    ])

    return pipeline



### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [62]:
def display_results(y_test, y_pred):
    labels = np.unique(y_pred)
    #confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    #print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)


In [63]:
X, y, category_names  = load_data()
X_train, X_test, y_train, y_test = train_test_split(X, y)

model = build_model()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

#display_results(y_test, y_pred)



### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [64]:

print(classification_report(y_test, y_pred, target_names=category_names))

ValueError: Unknown label type: (       related  request  offer  aid_related  medical_help  medical_products  \
22385        0        0      0            0             0                 0   
16483        0        0      0            0             0                 0   
14426        0        0      0            0             0                 0   
25563        1        0      0            0             0                 0   
25920        1        0      0            1             0                 0   
10837        1        1      0            0             0                 0   
15148        1        0      0            0             0                 0   
1208         1        1      0            1             0                 0   
2948         1        1      0            1             1                 0   
6199         2        0      0            0             0                 0   
12107        1        0      0            0             0                 0   
21648        1     

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.